## kNN classifier

In [4]:
import pandas as pd
url="https://coding.net/u/HongHuangNeu/p/Machine-Learning-Notes-Data/git/raw/master/DatingData/datingTestSet2.txt"
df=pd.read_csv(url,sep='\t')
df[:3]

,fre_flier_miles_earned_per_year,per_of_time_spent_playing_video_games,liters_of_ice_cream_consumed_per_year,label
0,40920,8.326976,0.953952,3
1,14488,7.153469,1.673904,2
2,26052,1.441871,0.805124,1
